# 2.5 ICESat-2 ATL08 30m Variant

Notebook that leverages PhoREAL repo to generate a 30m variant of the ATL08.  Using PhoREAL, it should be fairly easy to generate ATL08 of a varient of "bin" sizes.

## Import PhoREAL
# Note: how can we make the specific kernel needed for this notebook available?
ICESatReader and ICESatBinning need to be imported.  ICESatReader is read in to read the ATL03/ATL08 files by groundtrack.  ICSatBinning can be used to generate equalivalent new products at a variable "bin" length.  

In [8]:
# This chunk to be run if workspace has been restarted

#Create conda env required to run this nb
!conda create -n phorealpy python=3.7 -y
!conda activate phorealpy

# reqs (chk requirements.txt): pyproj, 
# pyproj version for PhoReal: > 2.0
!conda install pyproj h5py pandas scipy==1.5.2 -y
!conda install -c conda-forge laspy -y
!conda install -c conda-forge gdal -y

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/phorealpy

  added / updated specs:
    - python=3.7


The following NEW packages will be INSTALLED:

  _libgcc_mutex      pkgs/main/linux-64::_libgcc_mutex-0.1-main
  ca-certificates    pkgs/main/linux-64::ca-certificates-2021.1.19-h06a4308_0
  certifi            pkgs/main/linux-64::certifi-2020.12.5-py37h06a4308_0
  ld_impl_linux-64   pkgs/main/linux-64::ld_impl_linux-64-2.33.1-h53a641e_7
  libedit            pkgs/main/linux-64::libedit-3.1.20191231-h14c3975_1
  libffi             pkgs/main/linux-64::libffi-3.3-he6710b0_2
  libgcc-ng          pkgs/main/linux-64::libgcc-ng-9.1.0-hdf63c60_0
  libstdcxx-ng       pkgs/main/linux-64::libstdcxx-ng-9.1.0-hdf63c60_0
  ncurses            pkgs/main/linux-64::ncurses-6.2-he6710b0_1
  openssl            pkgs/main/linux-64::openssl-1.1.1i-h27cfd23_0
  pip                pkgs/main/linux-64::pip-20.3.3-py37h06a4308_0
  python             pkgs/main/linux-64::pytho

In [30]:
#!conda install scipy==1.5.2 -y
#!pip install scipy==1.5.2
!conda activate phorealpy


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [1]:
!conda remove pandas -y
!conda install pandas -y

Solving environment: - 
The environment is inconsistent, please check the package plan carefully
The following packages are causing the inconsistency:

  - defaults/linux-64::patsy==0.5.1=py37_0
  - defaults/linux-64::scikit-learn==0.20.1=py37hd81dba3_0
  - defaults/linux-64::scikit-image==0.14.1=py37he6710b0_0
/ 
  - r/linux-64::r-ps-1.3.0-r36h96ca727_0, r/linux-64::r-sys-3.2-r36h96ca727_0, r/noarch::r-numderiv-2016.8_1-r36h6115d3f_0, r/noarch::r-squarem-2017.10_1-r36h6115d3f_0
  - defaults/noarch::r-squarem-2017.10_1-r36h6115d3f_0, r/linux-64::r-ps-1.3.0-r36h96ca727_0, r/linux-64::r-sys-3.2-r36h96ca727_0, r/noarch::r-numderiv-2016.8_1-r36h6115d3f_0
  - defaults/noarch::r-numderiv-2016.8_1-r36h6115d3f_0, r/linux-64::r-ps-1.3.0-r36h96ca727_0, r/linux-64::r-sys-3.2-r36h96ca727_0, r/noarch::r-squarem-2017.10_1-r36h6115d3f_0
  - defaults/noarch::r-numderiv-2016.8_1-r36h6115d3f_0, defaults/noarch::r-squarem-2017.10_1-r36h6115d3f_0, r/linux-64::r-ps-1.3.0-r36h96ca727_0, r/linux-64::r-sys-3.

In [17]:
#!conda remove gdal -y
#!conda install gdal -y
!activate phorealpy
!conda env list
!conda activate phorealpy

# conda environments:
#
base                  *  /opt/conda
phorealpy                /opt/conda/envs/phorealpy
                         /projects/.local/share/r-miniconda
                         /projects/.local/share/r-miniconda/envs/r-reticulate


CommandNotFoundError: Your shell has not been properly configured to use 'conda activate'.
To initialize your shell, run

    $ conda init <SHELL_NAME>

Currently supported shells are:
  - bash
  - fish
  - tcsh
  - xonsh
  - zsh
  - powershell

See 'conda init --help' for more information and options.

IMPORTANT: You may need to close and restart your shell after running 'conda init'.




In [1]:
import sys
sys.path.insert(0, '../../../../above/gitlab_repos/PhoREAL/source_code')

from icesatReader import get_atl03_struct
from icesatReader import get_atl08_struct
from icesatBin import create_atl08_bin

affected functions: ['writeLas', 'writeHeaderFile']
affected functions: ['writeKml']


## Locate ATL03 and ATL08 files.

Locate the filepath or location of associated ATL03 and ATL08 files on MAAP.

In [27]:
# Still need to figure out exactly how to do this..

# As a place holder..

atl03filepath = '/projects/jabba/data/bbox_nsib1/ATL03_20190615091900_11930305_002_01.h5'
atl08filepath = '/projects/jabba/data/bbox_nsib1/ATL08_20190615091900_11930305_002_01.h5'

# And specifying a groundtrack
gt = 'gt1r'

## Read ATL03 and ATL08 files

Once files are located, they can be read in as PhoREAL structs.  ICESat Reader will create an ATL struct in which one of the attributes is a Pandas Dataframe with the contents of the height subgroup for a specified groundtrack.

In [28]:
# Read in the ATL03 Struct
'''
Inputs: the ATL03 file path, the groundtrack, and an associated ATL08 File Path (for photon-level classifications)
Output: An ATL03 Struct, with an associated pandas DF
'''
atl03 = get_atl03_struct(atl03filepath, gt, atl08filepath)

# Read in ATL08 Struct
'''
Inputs: The ATL08 File Path, a groundtrack, and the ATL03 struct (to match boundaries and extents)
Output: An ATL08 struct, with an associated pandas DF
'''
atl08 = get_atl08_struct(atl08filepath, gt, atl03)

## Rebin ATL08 to 100m to 30m

Read-bin ATL08 using ATL03 to generate a 30m ATL08 variant.  The ATL08 product includes geolocation attributes (longitude and latitude), height-related attributes related to ground and canopy cover, and flags related to ground conditions, collection attributes, and atmosphereic conditions.

In [29]:
atl08_30m = create_atl08_bin(atl03, atl08, res_at = 30)

# atl08_30m is a pandas dataframe and can be 
print(atl08_30m.columns)

ImportError: cannot import name 'roperator' from 'pandas.core.ops' (/opt/conda/lib/python3.7/site-packages/pandas/core/ops.py)

## Export bin to whatever 

The dataframe can be exported into the best format for processing.  H5 output similar to ATL08 is forth-coming into PhoREAL.  Also, this can be changed to be optimized however best it will be to process this on MAAP.  For now, we can export the output into a .csv.  

In [28]:
# Define an output location
output_csv = '/projects/jabba/data/bbox_nsib1/ATL08_30m_20190615091900_11930305_002_01.csv'

# Output DF to CSV
atl08_30m.to_csv(output_csv)

NameError: name 'atl0308_30m' is not defined